# Load dependencies

In [166]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
# from tensorflow.compact.v1 import ConfigProto
import matplotlib as plot

In [167]:
gpus=tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [168]:
cd D:/Course/DrivenData Project

D:\Course\DrivenData Project


# Import Files


In [169]:
data_dir='Dataset'
t=os.path.join(data_dir,'train_features')
p=os.listdir(os.path.join(data_dir,'train_features'))
len(p)

16488

In [170]:
df=pd.read_csv(os.path.join(data_dir,'train_features.csv'))
df

,id,filepath,site
0,ZJ000000,train_features/ZJ000000.jpg,S0120
1,ZJ000001,train_features/ZJ000001.jpg,S0069
2,ZJ000002,train_features/ZJ000002.jpg,S0009
3,ZJ000003,train_features/ZJ000003.jpg,S0008
4,ZJ000004,train_features/ZJ000004.jpg,S0036
...,...,...,...
16483,ZJ016483,train_features/ZJ016483.jpg,S0093
16484,ZJ016484,train_features/ZJ016484.jpg,S0043
16485,ZJ016485,train_features/ZJ016485.jpg,S0089
16486,ZJ016486,train_features/ZJ016486.jpg,S0095


In [171]:
train_labels = pd.read_csv(os.path.join(data_dir,'train_labels.csv'))
classes = train_labels.columns[1:]
t_labels = train_labels.iloc[:, 1:]
train_y = np.array([np.argmax(t_labels.iloc[i, :]) for i in range(t_labels.shape[0])])
l=list(train_y)
df['label']=l
df['label'].astype(dtype=str)

0        1
1        6
2        1
3        6
4        5
        ..
16483    2
16484    5
16485    3
16486    1
16487    3
Name: label, Length: 16488, dtype: object

In [172]:
pf=df.set_index('id').join(train_labels.set_index('id'))
pf

,filepath,site,label,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent
id,,,,,,,,,,,
ZJ000000,train_features/ZJ000000.jpg,S0120,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ZJ000001,train_features/ZJ000001.jpg,S0069,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ZJ000002,train_features/ZJ000002.jpg,S0009,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
ZJ000003,train_features/ZJ000003.jpg,S0008,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ZJ000004,train_features/ZJ000004.jpg,S0036,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
ZJ016483,train_features/ZJ016483.jpg,S0093,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
ZJ016484,train_features/ZJ016484.jpg,S0043,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
ZJ016485,train_features/ZJ016485.jpg,S0089,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [173]:
pf.columns

Index(['filepath', 'site', 'label', 'antelope_duiker', 'bird', 'blank',
       'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent'],
      dtype='object')

In [174]:
d=pf.astype(str,copy=False)
d
d[['path','file']] = d.filepath.str.split("/",expand=True)
d

,filepath,site,label,antelope_duiker,bird,blank,civet_genet,hog,leopard,monkey_prosimian,rodent,path,file
id,,,,,,,,,,,,,
ZJ000000,train_features/ZJ000000.jpg,S0120,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,train_features,ZJ000000.jpg
ZJ000001,train_features/ZJ000001.jpg,S0069,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,train_features,ZJ000001.jpg
ZJ000002,train_features/ZJ000002.jpg,S0009,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,train_features,ZJ000002.jpg
ZJ000003,train_features/ZJ000003.jpg,S0008,6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,train_features,ZJ000003.jpg
ZJ000004,train_features/ZJ000004.jpg,S0036,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,train_features,ZJ000004.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZJ016483,train_features/ZJ016483.jpg,S0093,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train_features,ZJ016483.jpg
ZJ016484,train_features/ZJ016484.jpg,S0043,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,train_features,ZJ016484.jpg
ZJ016485,train_features/ZJ016485.jpg,S0089,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train_features,ZJ016485.jpg


In [188]:
import tensorflow_addons as tfa
def preprocessing(im):
    im=tfa.image.gaussian_filter2d(im,filter_shape=(3,3),sigma=0.3)
#     mean = tfa.image.mean_filter2d(img, filter_shape=15)
    return im
    
                                   
    

Map Image to its label

In [189]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train=ImageDataGenerator(rescale=1/255.0,
                         validation_split=0.05,
                         zoom_range=0.20,
                         horizontal_flip=True,
                         brightness_range=[-1.0,1.0],
                         preprocessing_function=preprocessing)

train_gen=train.flow_from_dataframe(d,
                                    t,
                                    x_col='file',
                                    y_col='label',
                                    batch_size=32,
                                    color_mode='rgb',
                                    class_mode='categorical',
                                    subset='training',
                                    target_size=(224, 224))

valid_gen=train.flow_from_dataframe(d,
                                    t,
                                    x_col='file',
                                    y_col='label',
                                    batch_size=32,
                                    color_mode='rgb',
                                    class_mode='categorical',
                                    subset='validation',
                                    target_size=(224, 224))

Found 15664 validated image filenames belonging to 8 classes.
Found 824 validated image filenames belonging to 8 classes.


In [190]:
type(train_gen)

keras.preprocessing.image.DataFrameIterator

In [191]:
# cd D:/Course/DrivenData Project/Dataset

# Model

In [220]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l1, l2, l1_l2

In [214]:
def model1():
    model=Sequential()
    model.add(Conv2D(42,(3,3),input_shape=(224,224,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(42,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(42,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    
    model.compile(loss='binary_crossentropy',optimizer='Adam' ,metrics=['accuracy'])
    return model

In [203]:
model=model1()

In [204]:
type(model)

keras.engine.sequential.Sequential

In [205]:
model.fit_generator(train_gen,
          validation_data = train_gen,
          steps_per_epoch = train_gen.n//train_gen.batch_size,
          validation_steps = valid_gen.n//valid_gen.batch_size,
          epochs=15)

C:\Users\Karan\AppData\Local\Temp\ipykernel_14212\2364167053.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen,


Epoch 1/15
489/489 [==============================] - 222s 452ms/step - loss: 0.3659 - accuracy: 0.1977 - val_loss: 0.3569 - val_accuracy: 0.2138
Epoch 2/15
489/489 [==============================] - 214s 437ms/step - loss: 0.3519 - accuracy: 0.2247 - val_loss: 0.3466 - val_accuracy: 0.2587
Epoch 3/15
489/489 [==============================] - 209s 427ms/step - loss: 0.3448 - accuracy: 0.2565 - val_loss: 0.3392 - val_accuracy: 0.2650
Epoch 4/15
489/489 [==============================] - 259s 530ms/step - loss: 0.3396 - accuracy: 0.2619 - val_loss: 0.3312 - val_accuracy: 0.2875
Epoch 5/15
489/489 [==============================] - 221s 452ms/step - loss: 0.3326 - accuracy: 0.2835 - val_loss: 0.3263 - val_accuracy: 0.3013
Epoch 6/15
489/489 [==============================] - 222s 454ms/step - loss: 0.3280 - accuracy: 0.3004 - val_loss: 0.3268 - val_accuracy: 0.3250
Epoch 7/15
489/489 [==============================] - 223s 456ms/step - loss: 0.3248 - accuracy: 0.3063 - val_loss: 0.3156 -

In [85]:
# tf=model.fit(data,epochs=10,verbose=1,shuffle=True)

Evaluation


In [211]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 222, 222, 42)      1176      
                                                                 
 activation_45 (Activation)  (None, 222, 222, 42)      0         
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 111, 111, 42)     0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 109, 109, 42)      15918     
                                                                 
 activation_46 (Activation)  (None, 109, 109, 42)      0         
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 54, 54, 42)       0         
 g2D)                                                

In [206]:
score = model.evaluate_generator(valid_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\Karan\AppData\Local\Temp\ipykernel_14212\354537384.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(valid_gen)


Test loss: 0.3129803240299225
Test accuracy: 0.3507281541824341


In [209]:
model.save("CNN2.sav")

INFO:tensorflow:Assets written to: CNN2.sav\assets


INFO:tensorflow:Assets written to: CNN2.sav\assets


In [210]:
loaded=tf.keras.models.load_model('CNN2.sav')

In [230]:
def model2():
    model =Sequential()
    # Feature Extraction Section (The Convolution and The Pooling Layer)
    model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(224, 224, 3)))
    model.add(AveragePooling2D())
    model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
    model.add(AveragePooling2D())
    # Reshape the image into one-dimensional vector
    model.add(Flatten())
    # Classification Section (The Fully Connected Layer)
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(8, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['accuracy'])
    
    return model

In [231]:
model=model2()
type(model)

keras.engine.sequential.Sequential

In [232]:
model.fit(train_gen,
          validation_data = train_gen,
          steps_per_epoch = train_gen.n//train_gen.batch_size,
          validation_steps = valid_gen.n//valid_gen.batch_size,
          epochs=15)

Epoch 1/15
489/489 [==============================] - 209s 425ms/step - loss: 1.9458 - accuracy: 0.2140 - val_loss: 2.0421 - val_accuracy: 0.1825
Epoch 2/15
489/489 [==============================] - 204s 418ms/step - loss: 1.8866 - accuracy: 0.2368 - val_loss: 1.8686 - val_accuracy: 0.2812
Epoch 3/15
489/489 [==============================] - 202s 413ms/step - loss: 1.8536 - accuracy: 0.2492 - val_loss: 1.8161 - val_accuracy: 0.2950
Epoch 4/15
489/489 [==============================] - 210s 430ms/step - loss: 1.8095 - accuracy: 0.2707 - val_loss: 1.8093 - val_accuracy: 0.2525
Epoch 5/15
489/489 [==============================] - 202s 413ms/step - loss: 1.7733 - accuracy: 0.2881 - val_loss: 1.7201 - val_accuracy: 0.3100
Epoch 6/15
489/489 [==============================] - 201s 412ms/step - loss: 1.7663 - accuracy: 0.2932 - val_loss: 1.7471 - val_accuracy: 0.2875
Epoch 7/15
489/489 [==============================] - 203s 414ms/step - loss: 1.7368 - accuracy: 0.3058 - val_loss: 1.7417 -

In [233]:
score = model.evaluate_generator(valid_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\Karan\AppData\Local\Temp\ipykernel_14212\354537384.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(valid_gen)


Test loss: 1.6289719343185425
Test accuracy: 0.34951457381248474


In [234]:
model.save("LuNET.sav")

INFO:tensorflow:Assets written to: LuNET.sav\assets


INFO:tensorflow:Assets written to: LuNET.sav\assets


In [31]:
test=pd.read_csv('test_features.csv')
test

,id,filepath,site
0,ZJ016488,test_features/ZJ016488.jpg,S0082
1,ZJ016489,test_features/ZJ016489.jpg,S0040
2,ZJ016490,test_features/ZJ016490.jpg,S0040
3,ZJ016491,test_features/ZJ016491.jpg,S0041
4,ZJ016492,test_features/ZJ016492.jpg,S0040
...,...,...,...
4459,ZJ020947,test_features/ZJ020947.jpg,S0086
4460,ZJ020948,test_features/ZJ020948.jpg,S0152
4461,ZJ020949,test_features/ZJ020949.jpg,S0162
4462,ZJ020950,test_features/ZJ020950.jpg,S0055


In [37]:
img_size=(224,224)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [38]:
def make_gens(batch_size,test_df,img_size):
    t_gen=ImageDataGenerator()
    test_gen=t_gen.flow_from_dataframe(test,x_col='filepath',batch_size=32,class_mode=None,target_size=(224,224))
    test_steps=int(len(test_df)/batch_size)
    return test_gen,test_steps

In [39]:
test_gen,steps=make_gens(32,test,img_size)

Found 4464 validated image filenames.


In [40]:
test_gen[0].shape

(32, 224, 224, 3)

In [41]:
preds=model.predict(test_gen,verbose=1)

140/140 [==============================] - 33s 235ms/step


In [42]:
type(preds)
preds.shape
preds

array([[0.0000000e+00, 0.0000000e+00, 2.4538881e-24, ..., 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [43]:
np.set_printoptions(suppress=True)

In [44]:
preds[1]

array([0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [45]:
# preds_1 = np.array([np.argmax(preds[i, :]) for i in range(preds.shape[0])])

In [46]:
# preds

In [47]:
z_pred=np.zeros_like(preds)
for i in range(preds.shape[0]):
    j=np.argmax(preds[i,:])
    z_pred[i,j]=1
z_pred

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [48]:
final_labels=[]
for i in range(preds.shape[0]):
    final_labels.append(np.argmax(preds[i]))

In [49]:
final_labels=np.array(final_labels)

In [118]:
final_labels[0]

3

In [88]:
pp=np.unique(final_labels)

In [87]:
classes

Index(['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard',
       'monkey_prosimian', 'rodent'],
      dtype='object')

In [90]:
c_l=dict()
for i in pp:
    c_l[i]=classes[i]
c_l

{0: 'antelope_duiker',
 1: 'bird',
 2: 'blank',
 3: 'civet_genet',
 4: 'hog',
 5: 'leopard',
 6: 'monkey_prosimian',
 7: 'rodent'}

In [91]:
pp=np.unique(final_labels,return_counts=True)

In [93]:
pp[0]

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [94]:
for i in range(pp[0].shape[0]):
    pp[0][i]=np.where(pp[0,i] in c_l.keys(),
                      c_l[i],
                      0)

TypeError: tuple indices must be integers or slices, not tuple

In [51]:
sub=pd.read_csv('submission_format.csv')
sub=sub.iloc[:,1:].to_numpy()
sub[0]

array([0.04823279, 0.18918532, 0.04491418, 0.19958797, 0.10611783,
       0.13291451, 0.16640995, 0.11263745])

In [52]:
soc=np.zeros_like(sub)
for i in range(sub.shape[0]):
    j=np.argmax(sub[i,:])
    soc[i,j]=1
soc
# k=np.argmax(sub[0,:])
# k

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(z_pred, soc)

ModuleNotFoundError: No module named 'sklearn'

In [54]:
z_pred

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [55]:
soc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [56]:
np.savetxt('1',z_pred,delimiter=',')

In [ ]:
np.